# Overview from last week

### Interpolation
Say our we have measurements of fuction $f(x)$ at a discrete set of N points $x_i$, i = 1, N.

Interpolation answers the question: *What is the value of $f$ at any point $x$?*

To get to the answer we can make any of the following assumptions:
* $f$ is constant between two consecutive $x_i$ (nearest-neighbor interpolation)
* $f$ varies linearly between two consecutive $x_i$ (linear interpolation)
* $f$ can be described by a polynomial of some degree, which we are free to select (polynomial interpolation)
* $f$ between all pairs of $x_i, x_{i+1}$ is a polynomial of some degree and it and its derivative are continuous at all $x_i$ (spline interpolation)

### Non-parametric density estimation: Histograms

"How likely is it for a measurement to lie within a given interval of values?", or, "What is the distribution of the data?" If we have no reason to expect one of the usual models (with parameters, such as a Gaussian) to answer this question, then we can use non-parametric models. 

NOTE: "non-parametric" does not mean there are "no parameters". It means that in the method there is no *specific* class of functions (distributions) that describes the data (e.g. all possible Gaussians). 
In general, a "non-parametric" method:
* does not assume a priori that the data follow some functional form
* gives an estimate of the distribution function that approaches the true distribution with enough data.

The most-used non-parametric method is the *histogram* (parameters: bin size or number of bins and where the bin begins). The assumption of the method is that the true distribution function is constant within a bin. This excludes functions e.g. with infinite spikes, but otherwise is very generic. 

#### Best-choice for bin size?
The selection of the bin size is critical. It can make the difference between a single peak or multiple peaks in the distribution. The choice of $\Delta$ reflects a trade-off between having enough measurements in any given bin (minimizing noise) and having enough resolution to capture the important features of the pdf.

Bin size: $\Delta = \frac{max_x - min_x}{M}$, M is the number of bins.

Ideally, the bin size should take into account the number of datapoints, $N$. A number of methods can be used to select the bin size in such a way. 

One kind of method finds the optimal bin size by minimizing some function, usually this function is the Mean Integrated Squared Error (MISE):
$$ MISE = E[\int (h(x) - f(x))^2 dx], $$ 
where E denotes the expected value, $h(x)$ is the histogram, $f(x)$ is the true distribution and the integral runs over the range of values in the data (see e.g. Freedman-Diaconis 1981).
Since $f(x)$ is in general unknown, one can make assumptions about its form to find the optimal $\Delta$:
* Freedman-Diaconis 1981: assume $f$, $f'$ are continuous and $f'$ is bounded and vanishes at $\infty$. Also, $\int_I f'(x)^2 dx > 0$. Then $\Delta$ which minimizes the MISE is $\alpha N^{-1/3}$ and the IQR seems to be a good choice for $\alpha$ [paper: http://bayes.wustl.edu/Manual/FreedmanDiaconis1_1981.pdf]. *Good for* unimodal data (with outliers). *Bad for* e.g. uniform distribution, multimodal distributions.
* Scott 1992: assume the underlying distribution $f(x)$ is Gaussian: $\Delta = \frac{3.5 \sigma}{N^{1/3}}$. *Good for* distributions like the Normal?
* Shimazaki, Shinomoto 2007: assume each data sample follows a poisson random process (samples are independent). Make a histogram with M bins and compute the average bin height $k$ and the (biased) standard deviation of bin heights $v = \frac{1}{M}\sum_i=1^M (k_i - k)^2 $. Then vary the bins until you find that which corresponds to a width $\Delta$ that minimizes: $$C(\Delta) = \frac{2 k - v}{\Delta^2}$$ [This last equation is just a simplified expression for the MISE shown above, explanation: http://www.neuralengine.org//res/histogram.html].

Another kind of method relies on Bayesian inference and maximizes some other type of function. For example, Knuth 2006 assumes a piecwise constant density function (with equally-sized bins) for the true distribution and finds the most likely such model. Bayesian blocks is another such method, but allows for varying bin sizes. We'll revisit these after we talk about Bayesian inference...

NOTE: It should be clear that the bin size has an absolute minimum: it should always be larger than the data resolution. The above methods generally do not guarantee that the optimal bin size returned complies with this!

#### Exercise 1: Using data from the Kepler mission, these researchers [1] find an interesting result: The distribution of sizes of known exoplanets 'flattens off' in the range 1 - 3 Earth sizes! As put in this blog post [2]: "...planets with radii smaller than Neptune’s radius become more and more common but this increase in occurrence rate stops at some “critical” radius, after which the occurrence rate levels off (becomes flat). The reason why this is so important is that it implies that there is some aspect of planet formation and/or migration which is not scale-free. In other words, there is something special about a radius between 1 to 3 Earth radii." 
#### Using the methods mentioned above, let's check out the distribution of exoplanet sizes for ourselves and see if we think the result holds!

[1] http://adsabs.harvard.edu/abs/2013ApJ...770...69P
[2] For a blog post on the paper, see: http://exoplanetsdigest.com/2013/04/16/an-important-clue-for-earth-sized-planets-a-plateau-in-the-exoplanet-size-distribution/



In [17]:
# Load in libraries
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from astroML.plotting import hist as AMLhist

# Load in exoplanet data
datafull = np.genfromtxt('J_ApJ_770_69/table3.dat', delimiter = '|', usecols = 3)
data = data[data == data] # get rid of nans

In [23]:
# Make histograms using the different methods


In [24]:
# For the Shimazaki, Shinomoto method you can either write a function that makes
# a histogram following their prescription, or give the exoplanet data as input to the 
# ready-made python script histsample_torri.py, which came from this site: 
# http://www.neuralengine.org//res/histogram.html


### Non-parametric density estimation: Kernel Density Estimation (KDE)

The density at each point is the weighted mean of all points, with weights defined by the kernel function (typically they decrease with distance). 
For $N$ measurements $x_i$, the kernel density estimator is:
$$\hat{f}_N(x) = \frac{1}{N\Delta^D} \sum_{i=1}^N K(\frac{d(x,xi)}{\Delta})$$, where K is the kernel function, $\Delta$ is the size of the kernel and D is the number of dimensions. 

The kernel $K$ needs to be: 
* smooth and positive at all points 
* normalizable to unity
* centered around a zero mean and have a positive variance

A popular example is the Gaussian kernel: $K(u) = \frac{1}{(2\pi)^{D/2}} e^{-u^2/2}$

#### Best choice for kernel size?

Same as for histograms: you can minimize the MISE. 

The MISE can be re-written as:
$$ MISE = \int (E[h(x)] - f(x))^2 dx+ \int Var(h(x)) dx$$, i.e. the sum of the squared pointwise bias of the estimate and the pointwise variance. For KDE: $E[h(x)] = \int \frac{1}{\Delta} K(\frac{x-y}{\Delta} f(y) dy$ and $Var(h(x)) = \frac{1}{\Delta^2} K(\frac{x-y}{\Delta})^2 f(y) dy - [\frac{1}{\Delta} \int K(\frac{x-y}{\Delta}]^2$. 

To get some insight, let's assume our Kernel integrates to 1, is symmetric about 0, has a nonzero, finite second moment, and many continuous derivatives. Then it is shown [3] that the bias (approximately) depends on $\Delta^2$, while the variance on $N^{-1}\Delta^{-1}$. Therefore, to minimize the MISE we need to compromise between finding a small enough value of $\Delta$ to reduce the bias but also a large enough value to reduce the variance! 
Approximately optimal bandwidth:
$$ \Delta \approx N^{-1/5} k_2^{-2/5} [\int K^2(u)du]^{-1/5} [\int (f"(x))^2dx]^{-1/5}$$

#### Best choice for Kernel shape?

From the above equation it is clear that the optimal bandwidth depends on the kernel. The kernel that does the best job in finding the smallest MISE for this optimal $\Delta$ has the form of the positive part of a parabola (Epanechnikov kernel).

[3] https://www.stat.berkeley.edu/~stark/Teach/S240/Notes/ch10.pdf


#### Comparison to histograms: 
The error of the KDE using optimal bandwidth converges at a rate $N^{-4/5}$ (for large $N$). This is much faster than in the case of histograms where the convergence rate is $N^{-2/3}$. So KDE with optimal size is a better estimator of the true pdf compared to an optimized histogram (for large datasets). 

Proof: http://www2.stat.duke.edu/~zo2/shared/research/readings/kernelsmoothing.pdf

#### KDE with measurement errors
If the position of each point is measured with some uncertainty, then the true distribution is convolved with the distribution of the uncertainties and this results in the observed distribution. In this case, the *deconvolution* KDE method is used to get to the true distribution.

#### For censored data use the *weighted* KDE methods. A potentially useful application: Density estimation from Biased Sampling https://arxiv.org/pdf/0709.1616v1.pdf



#### Variable bandwidths: k-nearest neighbors
Instead of using a global bandwidth, we can use locally changing bandwidths $\Delta(x)$.
The general idea is to use a large bandwidth for regions where the data is sparse. The
k-nearest neighbor idea is to choose $\Delta(x) = d_K$, where $d_K$ is the Euclidean distance of x to the k
th nearest observation, where k is regulating the magnitude of the bandwidth. Note that generally,
the derived pdf will not be a density anymore since the integral is not necessarily equal to one.
The point density is: $$ \hat{f}_K(x) = \frac{K}{V_D(d_K)},$$ where $D$ is the number of dimensions.
The larger K, the more accurate the estimate but at the expense of the resolution. In practice should use at least K = 5 (otherwise variance too large).

The error of the method can be decreased by using the distances to all K nearest neighbors instead of just the distance to the K-th nearest neighbor (where C is a constant found by requiring $\hat{f}*$volume to be equal to the total number of data points):
$$ \hat{f}_K(x) = \frac{C}{\sum_{i=1}^K d_i^D} $$

## TODO:Some type of example here????
Also: 2D japanese method? Kernel based japanese method?

## The Curse of Dimensionality

Suppose our data are uniformly distributed on a 2-D plane within a unit square. Say we want to find an estimate of the density around some point, e.g. using a 2-D histogram. This means we will segment our 2-D square into smaller squares. For what size of the square bin ($s$), centered on some point, do we find 20% of the data within the bin? For uniformly sampled data, we can simply say that the area covered by the bin is equal to 0.2/(total area), so: $s = 0.2^{1/2}$. This can be generalized to any fraction $r$ of the data, and any number of dimensions, $D$: 
$$ s = r^{1/D}$$. 

reference: The Elements of Statistical Learning, Springer


#### Exercise: For D = 3,5,10... how far away does one need to go (how large a bin size) to capture a fraction $r$ of the data? (Best to demonstrate with a plot)

In [29]:
# Your code here

What we've discovered is one of the manifestations of the 'curse of dimensionality'. When the dimensions increase, the volume of the space increases so fast that the available data become sparse. Therefore, the number of points required to evenly sample the full data volume will grow exponentially with dimension!

Also, organizing and searching data often relies on detecting areas where objects form groups with similar properties; in high dimensional data, however, all objects appear to be sparse and dissimilar in many ways, which prevents common data organization strategies from being efficient.


But there is hope! Not all dimensions are equally important! Some do not have valuable information in them, so, if we identify them, we can only concentrate on the important ones... Dimensionality reduction techniques (e.g. Principal Component Analysis)...